<a href="https://colab.research.google.com/github/ruslanmv/watsonx-with-multimodal-llava/blob/master/4_Multimodal_Chat_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install gradio_multimodalchatbot
!pip install haversine
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 15.6 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-d5uaqqck
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-d5uaqqck
  Resolved https://github.com/huggingface/transformers.git to commit a22ff36e0e347d3d0095cccd931cbbd12b14e86a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9527412 sha256=7cd5844cdc3fcb2170852ebce1779196ef1c3142fad6ae90fe986fde2873ee68
  Stored in directory: /tmp/pip-ephem-wheel-cache-irdd4k7q/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found

In [1]:
import os
import pandas as pd
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import matplotlib.pyplot as plt
import urllib3
from transformers import pipeline
from transformers import BitsAndBytesConfig
import torch
import textwrap
import pandas as pd
import numpy as np
from haversine import haversine  # Install haversine library: pip install haversine
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)


model_id = "llava-hf/llava-1.5-7b-hf"

processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")


import os
import requests

url = 'https://github.com/ruslanmv/watsonx-with-multimodal-llava/raw/master/geocoded_hotels.csv'
filename = 'geocoded_hotels.csv'

# Check if the file already exists
if not os.path.isfile(filename):
    response = requests.get(url)

    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"File {filename} downloaded successfully!")
    else:
        print(f"Error downloading file. Status code: {response.status_code}")
else:
    print(f"File {filename} already exists.")

import os
import pandas as pd
from datasets import load_dataset
import pyarrow

# 1. Get the Current Directory
current_directory = os.getcwd()

# 2. Construct the Full Path to the CSV File
csv_file_path = os.path.join(current_directory, 'hotel_multimodal.csv')

# 3. Check if the file exists
if not os.path.exists(csv_file_path):
    # If not, download the dataset
    print("File not found, downloading from Hugging Face...")

    dataset = load_dataset("ruslanmv/hotel-multimodal")

    # Convert the 'train' dataset to a DataFrame using .to_pandas()
    df_hotels = dataset['train'].to_pandas()

    # 4.Save to CSV
    df_hotels.to_csv(csv_file_path, index=False)
    print("Dataset downloaded and saved as CSV.")


# 5. Read the CSV file
df_hotels = pd.read_csv(csv_file_path)

print("DataFrame loaded:")
geocoded_hotels_path = os.path.join(current_directory, 'geocoded_hotels.csv')
# Read the CSV file
geocoded_hotels = pd.read_csv(geocoded_hotels_path)

import requests

def get_current_location():
    try:
        response = requests.get('https://ipinfo.io/json')
        data = response.json()

        location = data.get('loc', '')
        if location:
            latitude, longitude = map(float, location.split(','))
            return latitude, longitude
        else:
            return None, None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

latitude, longitude = get_current_location()
if latitude and longitude:
    print(f"Current location: Latitude = {latitude}, Longitude = {longitude}")
else:
    print("Could not retrieve the current location.")


from geopy.geocoders import Nominatim

def get_coordinates(location_name):
    """Fetches latitude and longitude coordinates for a given location name.

    Args:
        location_name (str): The name of the location (e.g., "Rome, Italy").

    Returns:
        tuple: A tuple containing the latitude and longitude (float values),
               or None if the location is not found.
    """

    geolocator = Nominatim(user_agent="coordinate_finder")
    location = geolocator.geocode(location_name)

    if location:
        return location.latitude, location.longitude
    else:
        return None  # Location not found



def find_nearby(place=None):
    if place!=None:
        coordinates = get_coordinates(place)
        if coordinates:
            latitude, longitude = coordinates
            print(f"The coordinates of {place} are: Latitude: {latitude}, Longitude: {longitude}")
        else:
            print(f"Location not found: {place}")
    else:
        latitude, longitude = get_current_location()
        if latitude and longitude:
            print(f"Current location: Latitude = {latitude}, Longitude = {longitude}")
    # Load the geocoded_hotels DataFrame
    current_directory = os.getcwd()
    geocoded_hotels_path = os.path.join(current_directory, 'geocoded_hotels.csv')
    geocoded_hotels = pd.read_csv(geocoded_hotels_path)

    # Define input coordinates for the reference location
    reference_latitude = latitude
    reference_longitude = longitude

    # Haversine Distance Function
    def calculate_haversine_distance(lat1, lon1, lat2, lon2):
        """Calculates the Haversine distance between two points on the Earth's surface."""
        return haversine((lat1, lon1), (lat2, lon2))

    # Calculate distances to all other points in the DataFrame
    geocoded_hotels['distance_km'] = geocoded_hotels.apply(
        lambda row: calculate_haversine_distance(
            reference_latitude, reference_longitude, row['latitude'], row['longitude']
        ),
        axis=1
    )

    # Sort by distance and get the top 5 closest points
    closest_hotels = geocoded_hotels.sort_values(by='distance_km').head(5)

    # Display the results
    print("The 5 closest locations are:\n")
    print(closest_hotels)
    return closest_hotels


def search_hotel(place=None):
    import os
    import pandas as pd
    import requests
    from PIL import Image, UnidentifiedImageError
    from io import BytesIO
    import urllib3
    from transformers import pipeline
    from transformers import BitsAndBytesConfig
    import torch

    # Suppress the InsecureRequestWarning
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    # 1. Get the Current Directory
    current_directory = os.getcwd()
    # 2. Construct the Full Path to the CSV File
    csv_file_path = os.path.join(current_directory, 'hotel_multimodal.csv')
    # Read the CSV file
    df_hotels = pd.read_csv(csv_file_path)
    geocoded_hotels_path = os.path.join(current_directory, 'geocoded_hotels.csv')
    # Read the CSV file
    geocoded_hotels = pd.read_csv(geocoded_hotels_path)

    # Assuming find_nearby function is defined elsewhere
    df_found = find_nearby(place)

    # Converting df_found[["hotel_id"]].values to a list
    hotel_ids = df_found["hotel_id"].values.tolist()

    # Extracting rows from df_hotels where hotel_id is in the list hotel_ids
    filtered_df = df_hotels[df_hotels['hotel_id'].isin(hotel_ids)]

    # Ordering filtered_df by the order of hotel_ids
    filtered_df['hotel_id'] = pd.Categorical(filtered_df['hotel_id'], categories=hotel_ids, ordered=True)
    filtered_df = filtered_df.sort_values('hotel_id').reset_index(drop=True)

    # Define the quantization config and model ID
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    model_id = "llava-hf/llava-1.5-7b-hf"

    # Initialize the pipeline
    pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

    # Group by hotel_id and take the first 2 image URLs for each hotel
    grouped_df = filtered_df.groupby('hotel_id', observed=True).head(2)

    # Create a new DataFrame for storing image descriptions
    description_data = []

    # Download and generate descriptions for the images
    for index, row in grouped_df.iterrows():
        hotel_id = row['hotel_id']
        hotel_name = row['hotel_name']
        image_url = row['image_url']

        try:
            response = requests.get(image_url, verify=False)
            response.raise_for_status()  # Check for request errors
            img = Image.open(BytesIO(response.content))

            # Generate description for the image
            prompt = "USER: <image>\nAnalyze this image.  Give me feedback on whether this hotel is worth visiting based on the picture. Provide a summary  review.\nASSISTANT:"
            outputs = pipe(img, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
            description = outputs[0]["generated_text"].split("\nASSISTANT:")[-1].strip()

            # Append data to the list
            description_data.append({
                'hotel_name': hotel_name,
                'hotel_id': hotel_id,
                'image': img,
                'description': description
            })
        except (requests.RequestException, UnidentifiedImageError):
            print(f"Skipping image at URL: {image_url}")

    # Create a DataFrame from the description data
    description_df = pd.DataFrame(description_data)
    return description_df


def show_hotels(place=None):
    description_df = search_hotel(place)

    # Calculate the number of rows needed
    num_images = len(description_df)
    num_rows = (num_images + 1) // 2  # Two images per row

    fig, axs = plt.subplots(num_rows * 2, 2, figsize=(20, 10 * num_rows))

    current_index = 0

    for _, row in description_df.iterrows():
        img = row['image']
        description = row['description']

        if img is None:  # Skip if the image is missing
            continue

        row_idx = (current_index // 2) * 2
        col_idx = current_index % 2

        # Plot the image
        axs[row_idx, col_idx].imshow(img)
        axs[row_idx, col_idx].axis('off')
        axs[row_idx, col_idx].set_title(f"{row['hotel_name']}\nHotel ID: {row['hotel_id']} Image {current_index + 1}", fontsize=16)

        # Wrap the description text
        wrapped_description = "\n".join(textwrap.wrap(description, width=50))

        # Plot the description
        axs[row_idx + 1, col_idx].text(0.5, 0.5, wrapped_description, ha='center', va='center', wrap=True, fontsize=14)
        axs[row_idx + 1, col_idx].axis('off')

        current_index += 1

    # Hide any unused subplots
    total_plots = (current_index + 1) // 2 * 2
    for j in range(current_index, total_plots * 2):
        row_idx = (j // 2) * 2
        col_idx = j % 2
        if row_idx < num_rows * 2:
            axs[row_idx, col_idx].axis('off')
        if row_idx + 1 < num_rows * 2:
            axs[row_idx + 1, col_idx].axis('off')

    plt.tight_layout()
    plt.show()

def grouped_description(description_df):

  # Group by 'hotel_id' and aggregate descriptions
  grouped_descriptions = description_df.groupby('hotel_id')['description'].apply(lambda x: ' '.join(x.astype(str))).reset_index()

  # Merge with original DataFrame to get hotel names
  result_df = pd.merge(grouped_descriptions, description_df[['hotel_id', 'hotel_name']], on='hotel_id', how='left')

  # Drop duplicates and keep only the first occurrence of each hotel_id
  result_df = result_df.drop_duplicates(subset='hotel_id', keep='first')

  # Reorder columns
  result_df = result_df[['hotel_name', 'hotel_id', 'description']]
  return result_df

# prompt: please create a new python function that given the result_df as an input create a single prompt where  for given hotel_name you append the hotel_id and description , such we can use later this as context for a future llm query

def create_prompt_result(result_df):
  prompt = ""
  for _, row in result_df.iterrows():
    hotel_name = row['hotel_name']
    hotel_id = row['hotel_id']
    description = row['description']
    prompt += f"Hotel Name: {hotel_name}\nHotel ID: {hotel_id}\nDescription: {description}\n\n"
  return prompt
from transformers import pipeline, BitsAndBytesConfig
import torch
from langchain import PromptTemplate

# Create a LangChain prompt template for the hotel recommendation
hotel_recommendation_template = """
<s>[INST] <<SYS>>
You are a helpful and informative chatbot assistant.
<</SYS>>
Based on the following hotel descriptions, recommend the best hotel:
{context_result}
[/INST]
"""

# Use LangChain to create a prompt based on the template
def build_prompt(context_result):
    prompt_template = PromptTemplate(template=hotel_recommendation_template)
    return prompt_template.format(context_result=context_result)

# Quantization configuration for efficient model loading
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Initialize the text generation pipeline
pipe_text = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2",
                     model_kwargs={"quantization_config": quantization_config})

def generate_text_response(prompt):
    outputs = pipe_text(prompt, max_new_tokens=500)
    # Extract only the response after the instruction token
    response = outputs[0]['generated_text'].split("[/INST]")[-1].strip()
    return response
#place='Genova Italia'
#show_hotels(place)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

File geocoded_hotels.csv already exists.
DataFrame loaded:
Current location: Latitude = 36.175, Longitude = -115.1372


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
place='Genova Italia'

In [ ]:
description_df = search_hotel(place)

The coordinates of Genova Italia are: Latitude: 44.40726, Longitude: 8.9338624


<ipython-input-2-7e30f009360f>:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['hotel_id'] = pd.Categorical(filtered_df['hotel_id'], categories=hotel_ids, ordered=True)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


The 5 closest locations are:

       hotel_id                    hotel_name  chain_id  latitude  longitude  \
14386     19841          Bristol Palace Hotel        -1  44.40679    8.93620   
35138     48622       Best Western City Hotel         0  44.40925    8.93546   
34118     46980  Best Western Hotel Metropoli         0  44.40995    8.93501   
12274     16954               NH Genova Plaza        -1  44.41079    8.93791   
49601    432315    Old Port Genova Centro B&B        -1  44.41097    8.92867   

         city country    state  county      suburb postcode  \
14386  Genova  Italia  Liguria  Genova    Portoria    16123   
35138  Genova  Italia  Liguria  Genova    Portoria    16123   
34118  Genova  Italia  Liguria  Genova   Maddalena    16123   
12274  Genova  Italia  Liguria  Genova    Portoria    16122   
49601  Genova  Italia  Liguria  Genova  Centro Est    16100   

                      road house_number  distance_km  
14386  Via Venti Settembre    227 rosso     0.192905  


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Expanding inputs for image tokens in LLaVa should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.


Skipping image at URL: https://i.travelapi.com/hotels/1000000/10000/7400/7354/50ce3a3d_b.jpg
Skipping image at URL: https://i.travelapi.com/hotels/1000000/30000/22700/22623/9de3d2d2_b.jpg
Skipping image at URL: https://i.travelapi.com/hotels/13000000/12990000/12988500/12988492/a8199f02_b.jpg
Skipping image at URL: https://i.travelapi.com/hotels/13000000/12990000/12988500/12988492/7ef9ef38_b.jpg


In [ ]:
description_df

,hotel_name,hotel_id,image,description
0,Bristol Palace Hotel,19841,<PIL.JpegImagePlugin.JpegImageFile image mode=...,The image depicts a clean and well-maintained ...
1,Bristol Palace Hotel,19841,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"The image features a display case in a hotel, ..."
2,Best Western City Hotel,48622,<PIL.JpegImagePlugin.JpegImageFile image mode=...,The image features a hotel bathroom with two t...
3,Best Western Hotel Metropoli,46980,<PIL.JpegImagePlugin.JpegImageFile image mode=...,The image showcases a well-decorated hotel roo...
4,Best Western Hotel Metropoli,46980,<PIL.JpegImagePlugin.JpegImageFile image mode=...,The image features a hotel room with a large w...
5,NH Genova Plaza,16954,<PIL.JpegImagePlugin.JpegImageFile image mode=...,The image showcases a clean and well-organized...


In [ ]:
# Example usage:

result_df = grouped_description(description_df)
context_result=create_prompt_result(result_df)
recommendation_prompt = build_prompt(context_result)
result = generate_text_response(recommendation_prompt)
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Based on the descriptions and images provided, it is difficult to definitively recommend one hotel over the others without considering additional factors such as location, price, amenities, and guest reviews. However, here's a brief summary of each hotel based on the information given:

1. NH Genova Plaza (Hotel ID: 16954) - The image suggests a clean, well-maintained, and comfortable room with a fresh and inviting atmosphere.
2. Bristol Palace Hotel (Hotel ID: 19841) - The image showcases a luxurious and upscale bathroom with elegant decorative elements.
3. Best Western Hotel Metropoli (Hotel ID: 46980) - The image depicts a well-decorated and inviting room with a beautiful city view.
4. Best Western City Hotel (Hotel ID: 48622) - The image features a functional and well-maintained bathroom with two toilets and a shower.

To make an informed decision, it would be best to consider additional factors such as location, price, amenities, and guest reviews. You may also want to contact the

## Simple Chatbot with Multimodal only

In [12]:
import gradio as gr
from gradio_multimodalchatbot import MultimodalChatbot
from gradio.data_classes import FileData
#from backend import search_hotel

def get_hotel_recommendations(place):
    description_df = search_hotel(place)
    conversation = []
    for _, row in description_df.iterrows():
        hotel_name = row['hotel_name']
        description = row['description']
        img = row['image']

        # Save image temporarily to send as a file in chat
        img_path = f"{hotel_name}.png"
        img.save(img_path)

        user_msg = {"text": f"I want to see hotels in {place}.", "files": []}
        bot_msg = {
            "text": f"Here is {hotel_name}. {description}",
            "files": [{"file": FileData(path=img_path)}]
        }
        conversation.append([user_msg, bot_msg])

    return conversation

def chatbot_response(user_input):
    return get_hotel_recommendations(user_input)

with gr.Blocks() as demo:
    gr.Markdown("# Hotel Recommendation with Multimodal Chatbot")
    gr.Markdown("**Instructions**: Provide the place where you want to know which hotels you can visit.")

    conversation = []  # Initial empty conversation
    chatbot = MultimodalChatbot(value=conversation, height=800)

    with gr.Row():
        place_input = gr.Textbox(label="Enter a place")
        send_btn = gr.Button("Send")

    send_btn.click(chatbot_response, inputs=place_input, outputs=chatbot)

demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5203f6ed270e4b72f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The coordinates of Genova Italia are: Latitude: 44.40726, Longitude: 8.9338624


<ipython-input-2-1971a096bd7b>:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['hotel_id'] = pd.Categorical(filtered_df['hotel_id'], categories=hotel_ids, ordered=True)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


The 5 closest locations are:

       hotel_id                    hotel_name  chain_id  latitude  longitude  \
14386     19841          Bristol Palace Hotel        -1  44.40679    8.93620   
35138     48622       Best Western City Hotel         0  44.40925    8.93546   
34118     46980  Best Western Hotel Metropoli         0  44.40995    8.93501   
12274     16954               NH Genova Plaza        -1  44.41079    8.93791   
49601    432315    Old Port Genova Centro B&B        -1  44.41097    8.92867   

         city country    state  county      suburb postcode  \
14386  Genova  Italia  Liguria  Genova    Portoria    16123   
35138  Genova  Italia  Liguria  Genova    Portoria    16123   
34118  Genova  Italia  Liguria  Genova   Maddalena    16123   
12274  Genova  Italia  Liguria  Genova    Portoria    16122   
49601  Genova  Italia  Liguria  Genova  Centro Est    16100   

                      road house_number  distance_km  
14386  Via Venti Settembre    227 rosso     0.192905  


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Skipping image at URL: https://i.travelapi.com/hotels/1000000/10000/7400/7354/50ce3a3d_b.jpg
Skipping image at URL: https://i.travelapi.com/hotels/1000000/30000/22700/22623/9de3d2d2_b.jpg
Skipping image at URL: https://i.travelapi.com/hotels/13000000/12990000/12988500/12988492/a8199f02_b.jpg
Skipping image at URL: https://i.travelapi.com/hotels/13000000/12990000/12988500/12988492/7ef9ef38_b.jpg
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5203f6ed270e4b72f8.gradio.live


## Hybrid Chatbot with Multimodal and LLM

In [12]:
import gradio as gr
from gradio_multimodalchatbot import MultimodalChatbot
from gradio.data_classes import FileData
import time  # For simulating the search delay


def multimodal_results(description_df):
    conversation = []
    for _, row in description_df.iterrows():
        hotel_name = row['hotel_name']
        description = row['description']
        img = row['image']

        img_path = f"{hotel_name}.png"
        img.save(img_path)

        bot_msg = {
            "text": f"Here is {hotel_name}. {description}",
            "files": [{"file": FileData(path=img_path)}]
        }

        conversation.append([{"text": "", "files": []}, bot_msg])

    return conversation

def llm_results(description_df):
    result_df = grouped_description(description_df)
    context_result = create_prompt_result(result_df)
    recommendation_prompt = build_prompt(context_result)
    result = generate_text_response(recommendation_prompt)
    conversation = [[{"text": "Based on your search...", "files": []}, {"text": f"**My recommendation:** {result}", "files": []}]]
    return conversation


def chatbot_response(user_input, conversation):
    bot_initial_message = {
        "text": f"Looking for hotels in {user_input}...",
        "files": []
    }
    conversation.append([{"text": user_input, "files": []}, bot_initial_message])

    yield conversation

    description_df = search_hotel(user_input)

    if description_df is None or description_df.empty:
        error_message = {"text": f"Sorry, I couldn't find any hotels for {user_input}. Please try another location.", "files": []}
        conversation.append([{"text": user_input, "files": []}, error_message])
        yield conversation
        return  # Exit the function early

    hotel_conversation = multimodal_results(description_df)

    for message_pair in hotel_conversation:
        conversation.append(message_pair)
        yield conversation

    final_recommendation = llm_results(description_df)
    for message_pair in final_recommendation:
        conversation.append(message_pair)
        yield conversation


def initial_conversation():
    return [[
           {"text": "**Welcome to Hotel Recommendation!**", "files": []},
           {"text": "Please enter the place you're interested in visiting.", "files": []}
           ]]

with gr.Blocks() as demo:
    gr.Markdown("# 🏨 Hotel Recommendation Chatbot")
    gr.Markdown("**Provide the location to discover hotels and receive personalized recommendations!**")

    initial_conv = initial_conversation()
    chatbot = MultimodalChatbot(value=initial_conv, height=800)

    with gr.Row():
        place_input = gr.Textbox(label="Enter a place", placeholder="E.g., Paris, Tokyo, New York")
        send_btn = gr.Button("Search Hotels")

    send_btn.click(chatbot_response, inputs=[place_input, chatbot], outputs=chatbot)

demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://041ef2f28ab6c744a5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The coordinates of Genova,  Italia are: Latitude: 44.40726, Longitude: 8.9338624


<ipython-input-1-1971a096bd7b>:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['hotel_id'] = pd.Categorical(filtered_df['hotel_id'], categories=hotel_ids, ordered=True)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


The 5 closest locations are:

       hotel_id                    hotel_name  chain_id  latitude  longitude  \
14386     19841          Bristol Palace Hotel        -1  44.40679    8.93620   
35138     48622       Best Western City Hotel         0  44.40925    8.93546   
34118     46980  Best Western Hotel Metropoli         0  44.40995    8.93501   
12274     16954               NH Genova Plaza        -1  44.41079    8.93791   
49601    432315    Old Port Genova Centro B&B        -1  44.41097    8.92867   

         city country    state  county      suburb postcode  \
14386  Genova  Italia  Liguria  Genova    Portoria    16123   
35138  Genova  Italia  Liguria  Genova    Portoria    16123   
34118  Genova  Italia  Liguria  Genova   Maddalena    16123   
12274  Genova  Italia  Liguria  Genova    Portoria    16122   
49601  Genova  Italia  Liguria  Genova  Centro Est    16100   

                      road house_number  distance_km  
14386  Via Venti Settembre    227 rosso     0.192905  


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

<ipython-input-1-1971a096bd7b>:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['hotel_id'] = pd.Categorical(filtered_df['hotel_id'], categories=hotel_ids, ordered=True)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


The 5 closest locations are:

       hotel_id                        hotel_name  chain_id  latitude  \
17680     24265              Grand Hotel Arenzano        -1  44.40176   
30085     41228                  Hotel Puntabella        -1  44.37567   
14369     19820                      Hotel Palace        -1  44.36189   
10080     13969                     Hotel Riviera        -1  44.34332   
28233     38607  Best Western Premier CHC Airport        27  44.42131   

       longitude    city country    state  county         suburb postcode  \
17680    8.68544     NaN  Italia  Liguria  Genova      Borghetto    16011   
30085    8.62741     NaN  Italia  Liguria  Savona            NaN    16016   
14369    8.58361     NaN  Italia  Liguria  Savona            NaN    17019   
10080    8.54420     NaN  Italia  Liguria  Savona            NaN    17015   
28233    8.85326  Genova  Italia  Liguria  Genova  Medio Ponente    16153   

                          road house_number  distance_km  
17680    

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://041ef2f28ab6c744a5.gradio.live
